#1. Import Necessary Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

* numpy: For dataset creation and numerical operations.
* matplotlib.pyplot: For visualizing results.
* PolynomialFeatures: Expands input data into polynomial terms to simulate model complexity.
* Ridge: Implements Ridge regression with L2 regularization for controlled overfitting.
* mean_squared_error: To calculate the error between predicted and true values.
* train_test_split: Splits data into training and testing sets.


#2. Generate Synthetic Dataset

In [ ]:
# Generate synthetic dataset
np.random.seed(42)
n_samples = 30
X = np.random.uniform(-1, 1, size=(n_samples, 1))
y = np.sin(2 * np.pi * X).ravel() + 0.3 * np.random.normal(size=n_samples)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

* Dataset Creation:
  * X: Uniformly sampled points between -1 and 1.
  * y: Sine wave values with Gaussian noise (
𝜎
= 0.3) to simulate a noisy regression problem.
* Purpose: Mimics real-world data with inherent noise to analyze overfitting and regularization techniques.
* Train-Test Split:
  * 80% Training Data: Used to train the model.
  * 20% Testing Data: Used to evaluate generalization.

#3. Define Parameters

In [ ]:
# Parameters to explore
degrees = np.arange(1, 50)
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4]
alphas = [0.0, 0.1, 0.5, 1.0]  # L2 regularization levels

* degrees: Represents the polynomial degree, which determines model complexity.
* dropout_rates: Range of dropout rates to analyze their effect on overfitting and double descent.
* alphas: Ridge regression regularization strengths. Higher values enforce stronger weight penalization.

#4. Initialize Results Storage

In [ ]:
# Initialize data storage for results
results = {}

* A nested dictionary to store:
  * train_errors: Training errors for each combination of dropout and L2 regularization.
  * test_errors: Testing errors to assess generalization.

#5. Compute Results for Dropout and L2 Regularization

In [ ]:
for alpha in alphas:
    results[alpha] = {}
    for dropout_rate in dropout_rates:
        train_errors = []
        test_errors = []
        for degree in degrees:
            # Polynomial features
            poly = PolynomialFeatures(degree=degree)
            X_train_poly = poly.fit_transform(X_train)
            X_test_poly = poly.transform(X_test)

            # Apply dropout by randomly zeroing out features
            if dropout_rate > 0:
                mask = np.random.binomial(1, 1 - dropout_rate, X_train_poly.shape)
                X_train_poly = X_train_poly * mask

            # Ridge regression model
            model = Ridge(alpha=alpha)
            model.fit(X_train_poly, y_train)

            # Calculate errors
            y_train_pred = model.predict(X_train_poly)
            y_test_pred = model.predict(X_test_poly)
            train_errors.append(mean_squared_error(y_train, y_train_pred))
            test_errors.append(mean_squared_error(y_test, y_test_pred))

        # Store errors for current combination
        results[alpha][dropout_rate] = {
            "train_errors": train_errors,
            "test_errors": test_errors,
        }

1. Outer Loops:
* Iterate over all combinations of L2 regularization (alphas) and dropout rates (dropout_rates).
2. Inner Loop:
* Polynomial Feature Expansion:
Transforms input data into higher-dimensional space based on the degree.
* Dropout Simulation:
Randomly zeroes out features using a binary mask.
* Ridge Regression:
Fits the model using the regularized least squares approach.
* Error Calculation:
Compute Mean Squared Error (MSE) for both training and testing datasets.
3. Store Results:
* Errors for each combination are stored for later analysis and visualization.

#6. Visualize Results

In [ ]:
# Plot results
fig, axes = plt.subplots(len(alphas), 1, figsize=(10, 10), sharex=True)
for i, alpha in enumerate(alphas):
    ax = axes[i]
    for dropout_rate in dropout_rates:
        test_errors = results[alpha][dropout_rate]["test_errors"]
        ax.plot(
            degrees,
            test_errors,
            label=f"Dropout={dropout_rate}",
            marker="o",
            linestyle="-",
        )
    ax.set_yscale("log")
    ax.set_title(f"L2 Regularization (Alpha={alpha})")
    ax.set_xlabel("Model Complexity (Polynomial Degree)")
    ax.set_ylabel("MSE (Log Scale)")
    ax.legend()
    ax.grid()

plt.tight_layout()
plt.show()

1. Subplots:
* Each subplot represents a different L2 regularization strength (alpha).
* Dropout rates are compared within each subplot.
2. Plotting:
* X-Axis: Polynomial degree, representing model complexity.
* Y-Axis: Mean squared error in logarithmic scale.
3. Annotations:
* Titles, labels, and legends highlight L2 regularization effects and dropout rate variations.
